### Each_year_sharpe Ratio

In [15]:
Each_year_reault

{0: 7.174482255075866,
 1: 24.488556583419726,
 2: 10.959899074953276,
 3: 13.804995666429233,
 4: 10.8070808038845,
 5: 2.0055475561414697,
 6: 4.286516949898234,
 7: 13.80545836049619,
 8: 2.301452287391256,
 9: 0.016599429655902772,
 10: 3.1235787745247046,
 11: 8.565358013794883,
 12: 16.60309507217261,
 13: 4.349747574847736,
 14: 2.778485498329794,
 15: 8.98429055268362,
 16: 59.02434874539888,
 17: 18.012661464710323,
 18: 4.937923993410727,
 19: 1.6258602783970342}

In [1]:
import pandas as pd
import numpy as np
import time
from math import sqrt

In [2]:
class platform:
    
    def __init__(self, filename, interest): #newsfile):
        close_price = pd.read_csv(filename)
        #news = pd.read_csv(newsfile)
        date = close_price['datetime']
        date_r = close_price['datetime'][1:] #the date use for calucalting return
        date_r = date_r.reset_index(drop=True)
        toreduce = close_price.iloc[:-1, 1:].copy()
        allprices = close_price.iloc[1:, 1:].copy()
        allprices = allprices.reset_index(drop=True)
        daily_return = allprices - toreduce
        percent_c = daily_return/toreduce
        daily_return['date'] = date_r
        percent_c['date'] = date_r 
        daily_return = daily_return.set_index('date')
        percent_c = percent_c.set_index('date')
        
        #self.news = news
        self.date = date
        self.close = close_price
        self.d_return = daily_return #in dolloar
        self.per_return = percent_c #in percent
        self.interest = interest
        
        
    
    
    def test_strategy1(self): #long stocks have most negative return before, short stocks have most positve return the day before.
        strategy = []
        for i in range(len(self.date)-1):  
            today = self.date[i+1]
            pre_return = self.per_return.loc[today]
            pre_return = pre_return.replace([np.inf, -np.inf, np.nan], 0) #replacing +- inf and nan with 0
            sort_return = pre_return.sort_values()
            short10 = list(sort_return[-10:].index) 
            long10 = list(sort_return[:10].index)
            for i in range(len(long10)): long10[i] = '+' + long10[i]
            for i in range(len(short10)): short10[i] = '-' + short10[i]
            strategy.append([long10+short10])
        addate = self.date[1:]
        addate = addate.reset_index(drop=True)
        dataframe = pd.DataFrame(strategy, columns = ['Pick'])
        dataframe['Date'] = addate
        dataframe = dataframe.set_index('Date')
        return dataframe
    
    def daily_return_per(self, time, day_pick): #time should be the next day for day_pick
        return_per = self.per_return.loc[time] #get the next_day return
        retu = 0
        #old = 0
        length = 0 #the numbers of stock we had trade
        #print(retu-return_per['CVU']) bug
        for i in day_pick: #if next day is nan or inf, will just pass. there are some data problem see below
            the_return =return_per[i[1:]]
            if np.isnan(return_per[i[1:]]) or np.isinf(the_return):
                pass
            else:
                length += 1 # successful return, not nan or inf
                if i[0] == '-': #we are shorting with - sign
                    retu -= the_return
                else:
                    retu += the_return
            # trying to solve the inf problem, not really good :(
            ## np.isinf(retu):
             #  print(old)
             #  print(return_per[i[1:]])
             #  print(retu)
             #  #print(return_per['CVU'])
             #  print(day_pick)
             #  print("bug")
             #  print(i)
             #  print(time)     
        #old = retu
        if length > 1: 
            retu = retu/(length/2) #just buying or selling in of length percent.
        return retu
        
        
    def get_all_return(self, stock_picks): #this will return all daily return in years dict form
        cur_index = 0 # to loop the correct dates
        return_per_year = {}
        start_year = int(str(stock_picks.index[0])[2:4]) #the year started on the stock_pick dataframe
        end_year = int(str(stock_picks.index[-1])[2:4]) #the year ended on the stock_pick dataframe
        for i in range(end_year-start_year+1): #return by the number of year used. 00-02, 3 year in total,00 01 02
            #print(i)
            per_day_return = [] #store all daily return into a list, 
            i += 1 + start_year #stop on the year+1, start 00 stop 01.
            if i <10:
                breaktime = int("200{}0101".format(i))
            else:
                breaktime = int("20{}0101".format(i))
            #print(breaktime)
            picks_time = stock_picks.index[cur_index:] # as we need to break when the year end, cur_index is needed.
            for ind in range(len(picks_time)-1): 
                pre_day = picks_time[ind]
                #print(stock_picks.loc[pre_day][0])
                #print(pre_day)
                #print(len(picks_time)-1)
                #print(ind)
                # getting pre_day + one day.
                
                #test.date[test.date[test.date == 20000103].index[0]+1]
                today = self.date[self.date[self.date == pre_day].index[0]+1] #this will allows us to get last day return of the pick
                if pre_day > breaktime:
                    #print('break')
                    break
                per_day_return.append(self.daily_return_per(today, stock_picks.loc[pre_day][0]))
                cur_index += 1
            return_per_year[i-1] = (per_day_return)
        return return_per_year
            
    def getsharp(self, stock_picks):
        sharp = {}
        all_return = self.get_all_return(stock_picks)
        for i in all_return: #for the ith year
            profit = 1
            std = sqrt(251*np.var(all_return[i]))
            if std == 0:
                std = 1
            #std = np.std(all_return[i]) old one
            #print(std)
            for each_return in all_return[i]: #each days return
                profit = profit*(1+each_return)
            #print('profit', profit)
            
            #sharpe_ratio = np.sqrt(252) * (np.mean(daily_returns) / np.std(daily_returns)) this from the shell codes
            sharp[i] = ((profit-1) - self.interest)/std
        return sharp
        #sharp = (Ruturn - free interest)/standard_dv
    
    def five_days_avgsc(self, news_scores): ##simpliy avgerage does not work well
        scores_frame1 = news_scores.loc[:,news_scores.columns.isin(list(self.close))]
        st1 = scores_frame1.reset_index()
        st2 = st1.drop(columns='Date')
        Date = st1['Date']
        st3 = st2.fillna(0)
        empty_1st_row = {}
        for i in list(st3): empty_1st_row[i] = 0
        empty_row = pd.DataFrame([empty_1st_row])
        # will be adding st3, st4, st5, st6, st7. weight can be changed as well.
        st4 = pd.concat([empty_row, st3[:-1]],sort=False).reset_index(drop = True) 
        st5 = pd.concat([empty_row, empty_row, st3[:-2]],sort=False).reset_index(drop = True)
        st6 = pd.concat([empty_row, empty_row, empty_row, st3[:-3]],sort=False).reset_index(drop = True) 
        st7 = pd.concat([empty_row, empty_row, empty_row, empty_row, st3[:-4]],sort=False).reset_index(drop = True) 
        st8 = (st3+st4+st5+st6+st7)/5
        avg_frame =st8.set_index(Date)
        return avg_frame
    
    def five_days_weight(self, news_scores, w_list): ##simpliy avgerage does not work well
        scores_frame1 = news_scores.loc[:,news_scores.columns.isin(list(self.close))]
        st1 = scores_frame1.reset_index()
        st2 = st1.drop(columns='Date')
        Date = st1['Date']
        st3 = st2.fillna(0)
        empty_1st_row = {}
        for i in list(st3): empty_1st_row[i] = 0
        empty_row = pd.DataFrame([empty_1st_row])
        # will be adding st3, st4, st5, st6, st7. weight can be changed as well.
        st4 = pd.concat([empty_row, st3[:-1]],sort=False).reset_index(drop = True) 
        st5 = pd.concat([empty_row, empty_row, st3[:-2]],sort=False).reset_index(drop = True)
        st6 = pd.concat([empty_row, empty_row, empty_row, st3[:-3]],sort=False).reset_index(drop = True) 
        st7 = pd.concat([empty_row, empty_row, empty_row, empty_row, st3[:-4]],sort=False).reset_index(drop = True) 
        st8 = (st3*w_list[0]+st4*w_list[1]+st5*w_list[2]+st6*w_list[3]+st7*w_list[4])/5
        avg_frame =st8.set_index(Date)
        return avg_frame
    
    def test_strategy2(self, scores_frame, cap=0.8): #no moumantian, just simply scores*confidence
        #the scores_frame contains 60K tickers, but we only got 3k to trade
        strategy = []
        #this will select tickers only exists in news and stocks_closeprice.
        scores_frame1 = scores_frame.loc[:,scores_frame.columns.isin(list(self.close))]
        for i in range(len(self.date)-1):  
            today = self.date[i+1]
            if today == 20190301: #the news doesn't go after 0228
                break
            daily_sc = scores_frame1.loc[today].dropna().sort_values() #this will sort the value of scorces
            #long and short the same number of securites.
            #Assign cap so it will only pick up if scores is up.
            shorted_count = 0
            tolong = []
            toshort = []
            if len(daily_sc) > 20:
                for i in daily_sc.index[:10]: #this is sorted from negative to positive
                    if daily_sc[i] < -cap:  # if the scorces*confidence < -0.8, vaild
                        shorted_count += 1 
                        toshort.append(i)
                for i in daily_sc.index[-shorted_count:]: 
                    if daily_sc[i] > cap:  # if the scorces*confidence < -0.8, vaild
                        tolong.append(i)
            else: # there are less than 20 news scores at that day. opps
                number = len(daily_sc)/2
                for i in daily_sc.index[:number]: 
                    if daily_sc[i] < -cap:  
                        shorted_count += 1 
                        toshort.append(i)
                for i in daily_sc.index[-shorted_count:]:
                    if daily_sc[i] > cap:
                        tolong.append(i)
            #now we need to make sure there are same number of short and long
            max_num = min(len(tolong),len(toshort))
            tolong = tolong[-max_num:] # the last in tolong have the most positive value, oppiste of toshort
            toshort = toshort[:max_num]
            for i in range(len(tolong)): tolong[i] = '+' + tolong[i]
            for i in range(len(toshort)): toshort[i] = '-' + toshort[i]
            strategy.append([tolong+toshort])
        addate = self.date[1:]
        addate = addate.reset_index(drop=True)
        dataframe = pd.DataFrame(strategy, columns = ['Pick'])
        dataframe['Date'] = addate
        dataframe = dataframe.set_index('Date')
        return dataframe
    
#Taking average of five days scores.        

## Getting the news' Scores.

In [3]:
test = platform('final_clprice.csv', 0.02)

### Notes:
By comparing each news' source, DJN seems to be the most important news as returning the highest sharpe ratio. BEN and DJE is seems not good scources and will assign them as zero.
Tried log(1/Novelity + 1). The results seems worsen. 
Moreover, Five days' average or weighted average didn't work as well as the simple previous day scores. 
The scores selecting strategy will select equal amount of securities to short and long

In [4]:
def SourceFactor(thesource):
    if thesource == 'DJN':
        return 1.3
    elif thesource == 'DPR':
        return 1.0
    elif thesource == 'TPC':
        return 0.9
    elif thesource == 'BNZ':
        return 0.85
    elif thesource == 'WSJ':
        return 0.82
    elif thesource == 'BAR':
        return 0.8
    elif thesource == 'DWB':
        return 0.78
    else:
        return 0.5

In [5]:
# Some of the events seems very helpful in return, such as EST, INT, BUY, and GDC!!!
def EventsFactor(thesource):
    if thesource is not str:
        return 0.4
    if 'AA@GDC' in thesource:
        return 50
    elif 'AA@ERN' in thesource:
        return 20
    for news in ['AA@BUY', 'AA@EST', 'AA@INS', 'AA@RAT', 'AA@OPS', 'AA@FDA','AA@DIV']:
        if news in thesource:
            return 1.3
    for news2 in ['AA@REG','AA@MGT','AA@HFD','AA@GOV','AA@IPO','AA@REG', 'AA@OWN']:
        if news2 in thesource:
            return 1
    for bad in ['AA@AST','AA@ETF']:
        if bad in thesource:
            return 0.1
    if thesource == 'AA@ALEX':
        return 0.9
    else:
        return 0.4

In [6]:
news_frame = pd.read_csv('All_news.csv')

In [7]:
all_score = {} 
counter = 0
#sc_factor = [1.3, 1.0, 0.9, 0.85, 0.82, 0.80, 0.78]
#reducing the size of news_frame
all_ticker = list(test.close)[1:]
news_frame = news_frame.loc[news_frame['Ticker'].isin(all_ticker)]
reduced_news = news_frame[['Ticker','Sentiment(trinary score)','Confidence', 'adj_date','Source', 'Novelty','Events']]
news_date = list(set(news_frame['Date']))
news_date.sort()
tstart = time.time()
for each_day in news_date:
    counter += 1
    the_day = reduced_news.loc[reduced_news['adj_date'] == each_day]
    valid = the_day.loc[the_day['Sentiment(trinary score)'] != 0]
    the_dict = {}
    for i in range(len(valid)):
        item = valid.iloc[i]
        ticker = item['Ticker']
        if ticker in the_dict:
            the_dict[ticker] += item['Sentiment(trinary score)']* item['Confidence'] * SourceFactor(item['Confidence']) *EventsFactor(item['Events'])
        else:
            the_dict[ticker] = item['Sentiment(trinary score)']* item['Confidence'] * SourceFactor(item['Confidence']) *EventsFactor(item['Events'])
    all_score[each_day] = the_dict
    if counter%100 == 0:
        print(each_day)
        tend=time.time()
        print(tend - tstart)
news_fin = pd.DataFrame(all_score).T
news_fin = news_fin.rename(columns={'Unnamed: 0': "Date"})
news_fin.to_csv('news_scores_fin1.0.csv')

20000410
14.416305541992188
20000719
31.825672388076782
20001027
47.66744112968445
20010205
59.04209041595459
20010516
82.79184365272522
20010824
103.28843188285828
20011202
121.60069537162781
20020312
136.1636106967926
20020620
150.53041887283325
20020928
164.53748440742493
20030106
177.07373714447021
20030416
190.11580991744995
20030725
205.75137639045715
20031102
221.33067965507507
20040210
240.67770338058472
20040520
271.91797518730164
20040828
298.8616449832916
20041206
329.11358642578125
20050316
358.5503833293915
20050624
388.12749671936035
20051002
418.9046938419342
20060110
448.99166536331177
20060420
481.27246856689453
20060729
510.71132612228394
20061106
539.9428372383118
20070214
568.2068314552307
20070525
597.8286020755768
20070902
624.8426349163055
20071211
652.3793015480042
20080320
678.7337110042572
20080628
708.8205122947693
20081006
737.6275100708008
20090114
767.6325056552887
20090424
799.275981426239
20090802
827.4740023612976
20091110
862.5974271297455
20100218
896

## Sharp Ratio of Each Year

This Sharpe Ratio is only based on Sentiment and Confidence.
More experiment on below cells

In [13]:
sta3 = test.test_strategy2(news_fin, cap=0.7)

In [14]:
Each_year_reault = test.getsharp(sta3)

In [45]:
sum(test.getsharp(sta3).values()) #0.6

217.65593893561598

In [52]:
test.getsharp(sta3)

195.41388362070774

# Example of close price and percentage Return

### closing price

In [6]:
test.close[:3]

,datetime,A,AA,AABA,AAI,AAL,AAN,AAOI,AAON,AAP,...,ZION,ZIXI,ZLC,ZLTQ,ZMH,ZNT,ZOLL,ZQK,ZTS,ZUMZ
0,20000103,NaN,121.400180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.5000,37.1875,23.5,NaN,NaN,13.583333,NaN,3.703125,NaN,NaN
1,20000104,NaN,121.962652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.8125,36.5000,22.5,NaN,NaN,13.375000,NaN,3.546875,NaN,NaN
2,20000105,NaN,128.993550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.7500,37.2813,23.0,NaN,NaN,13.500000,NaN,3.484375,NaN,NaN


### percentage return of that day (while buy on the previous day) if buy on 0104, the return is in 0105

In [7]:
#percentage return of that day (while buy on the previous day)
#if buy on 0104, the return is in 0105
test.per_return[:3]

,A,AA,AABA,AAI,AAL,AAN,AAOI,AAON,AAP,AAPL,...,ZION,ZIXI,ZLC,ZLTQ,ZMH,ZNT,ZOLL,ZQK,ZTS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
20000104,NaN,0.004633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.084310,...,-0.048423,-0.018487,-0.042553,NaN,NaN,-0.015337,NaN,-0.042194,NaN,NaN
20000105,NaN,0.057648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014634,...,-0.001183,0.021405,0.022222,NaN,NaN,0.009346,NaN,-0.017621,NaN,NaN
20000106,NaN,-0.013081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.086538,...,0.014218,-0.054486,-0.043478,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN


### Catagorizing different news type file

In [58]:
news_frame = pd.read_csv('All_news.csv')

In [30]:
all_type = set(news_frame['Events'][:].dropna())
all_type = list(all_type)
unique_type = []
for i in all_type:
    for k in i.split(','):
        unique_type.append(k)
unique_type = set(unique_type) 

In [62]:
all_type

['AA@ALEX,AA@CPG,AA@DEB,AA@MGT',
 'AA@ALEX,AA@DEB,AA@REG',
 'AA@ALEX,AA@FDA,AA@REG,AA@TRD',
 'AA@ALEX,AA@BUY,AA@MNA,AA@OPS',
 'AA@ALEX,AA@ERN,AA@MNA,AA@PRE',
 'AA@ALEX,AA@ERN,AA@EST,AA@PRE,AA@RAT',
 'AA@ALEX,AA@DEB,AA@DIV,AA@ERN',
 'AA@ALEX,AA@FDA,AA@IPO,AA@OPS',
 'AA@ALEX,AA@DIV,AA@EST,AA@MGT',
 'AA@ALEX,AA@INS,AA@IPO',
 'AA@ALEX,AA@ERN,AA@EST',
 'AA@ALEX,AA@DIV,AA@IPO',
 'AA@ALEX,AA@ACC,AA@OPS,AA@OWN',
 'AA@ALEX,AA@CPG,AA@DIV,AA@OPS',
 'AA@ALEX,AA@GOV,AA@OPS',
 'AA@ALEX,AA@ERN,AA@EST,AA@GDC,AA@TRD',
 'AA@ALEX,AA@MFD,AA@MNA,AA@OPS,AA@PRE',
 'AA@ALEX,AA@DEB,AA@MNA,AA@REG',
 'AA@ALEX,AA@DRV,AA@ERN,AA@FDA',
 'AA@ALEX,AA@FDA,AA@MNA,AA@OPS',
 'AA@ALEX,AA@ERN,AA@GDC,AA@MGT,AA@OPS',
 'AA@ALEX,AA@ACC,AA@MGT',
 'AA@ALEX,AA@ACC,AA@FDA,AA@OPS,AA@REG',
 'AA@ALEX,AA@CPA,AA@ERN,AA@GDC,AA@MGT',
 'AA@ALEX,AA@DIV,AA@ERN,AA@MFD',
 'AA@ALEX,AA@CPG,AA@LGL,AA@MNA',
 'AA@ALEX,AA@CPG,AA@OPS,AA@TRD',
 'AA@ALEX,AA@DEB,AA@ERN,AA@GOV,AA@OPS',
 'AA@ALEX,AA@ERN,AA@FDA',
 'AA@ALEX,AA@CPG,AA@ERN,AA@MGT',
 'AA@ALEX,

In [41]:
unique_news = list(unique_type)
unique_news.remove('AA@ALEX')

In [44]:
unique_news

['AA@BNQ',
 'AA@ERN',
 'AA@IMB',
 'AA@OPS',
 'AA@CPA',
 'AA@GOV',
 'AA@LGL',
 'AA@OWN',
 'AA@GDC',
 'AA@DRV',
 'AA@EST',
 'AA@MGT',
 'AA@IPO',
 'AA@HFD',
 'AA@RAT',
 'AA@MNA',
 'AA@ACC',
 'AA@INS',
 'AA@DMNA',
 'AA@MFD',
 'AA@BUY',
 'AA@CBT',
 'AA@FDA',
 'AA@PRE',
 'AA@DIV',
 'AA@ETF',
 'AA@DEB',
 'AA@AST',
 'AA@REG',
 'AA@TRD',
 'AA@CPG']

In [ ]:
#news_frame = pd.read('All_news.csv')
all_score = {}
#Source = ['DJE', 'BAR', 'DJN', 'BNZ', 'BEN', 'TPC', 'DPR', 'WSJ','DWB']
Source = ['TPC', 'DPR', 'WSJ','DWB']
#counter = 0
for thenewsc in Source:
    news_date = list(set(test.news['Date']))
    news_date.sort()
    tstart = time.time()
    for each_day in news_date:
        #counter += 1
        the_day = test.news.loc[test.news['adj_date'] == each_day,['Ticker','Sentiment(trinary score)','Confidence','Source']]
        source_news = the_day.loc[the_day['Source'] != thenewsc]
        valid = source_news.loc[the_day['Sentiment(trinary score)'] != 0]
        the_dict = {}
        for i in range(len(valid)):
            item = valid.iloc[i]
            ticker = item['Ticker']
            if ticker in the_dict:
                the_dict[ticker] += item['Sentiment(trinary score)']*item['Confidence']
            else:
                the_dict[ticker] = item['Sentiment(trinary score)']*item['Confidence']
        all_score[each_day] = the_dict
        #if counter%365 == 0:
            #print(each_day)
            #tend=time.time()
            #print(tend - tstart)
    news_scores = pd.DataFrame(all_score).T
    #news_scores[news_scores.columns.dropna()]
    news_scores = news_scores.rename(columns={'Unnamed: 0': "Date"})
    news_scores.to_csv('news_scores_{}.csv'.format(thenewsc))

## Catagorizing different news sources file
### using different scores file to get sharpe and see which one performs well

['DJE', 'BAR', 'DJN', 'BNZ', 'BEN', 'TPC', 'DPR', 'WSJ','DWB']

In [5]:
news_frame = pd.read_csv('All_news.csv')
all_tickersn = list(test.close)[1:]
news_frame = news_frame.loc[news_frame['Ticker'].isin(all_tickersn)]

In [7]:
start_t = time.time()
all_score = {}
Source = ['DJE', 'BAR', 'DJN', 'BNZ', 'BEN', 'TPC', 'DPR', 'WSJ','DWB']
#counter = 0
for thenewsc in Source:
    start_each = time.time()
    news_date = list(set(news_frame['Date']))
    news_date.sort()
    tstart = time.time()
    source_news = news_frame[news_frame['Source'] == thenewsc]
    source_news = source_news[['Ticker','Sentiment(trinary score)','Confidence', 'adj_date']]
    for each_day in news_date:
        #counter += 1
        the_day = source_news[source_news['adj_date'] == each_day]
        valid = the_day[the_day['Sentiment(trinary score)'] != 0]
        the_dict = {}
        for i in range(len(valid)):
            item = valid.iloc[i]
            ticker = item['Ticker']
            if ticker in the_dict:
                the_dict[ticker] += item['Sentiment(trinary score)']*item['Confidence']
            else:
                the_dict[ticker] = item['Sentiment(trinary score)']*item['Confidence']
        all_score[each_day] = the_dict
        #if counter%365 == 0:
            #print(each_day)
            #tend=time.time()
            #print(tend - tstart)
    news_scores = pd.DataFrame(all_score).T
    news_scores = news_scores.rename(columns={'Unnamed: 0': "Date"})
    news_scores.to_csv('news_scores_{}.csv'.format(thenewsc))
    print(start_each - time.time())
print(start_t - time.time())

-137.71029257774353
-148.44589161872864
-478.0849690437317
-201.4144902229309
-139.02589178085327
-322.6413481235504
-193.55870246887207
-159.29969239234924
-182.42231941223145
-1962.6045761108398


In [23]:
DJE =pd.read_csv('news_scores_DJE.csv')
BAR =pd.read_csv('news_scores_BAR.csv')
DJN =pd.read_csv('news_scores_DJN.csv')
BNZ =pd.read_csv('news_scores_BNZ.csv')
BEN =pd.read_csv('news_scores_BEN.csv')
TPC =pd.read_csv('news_scores_TPC.csv')
DPR =pd.read_csv('news_scores_DPR.csv')
WSJ =pd.read_csv('news_scores_WSJ.csv')
DWB =pd.read_csv('news_scores_DWB.csv')

In [24]:
DJE = DJE.rename(columns={'Unnamed: 0': "Date"})
BAR = BAR.rename(columns={'Unnamed: 0': "Date"})
DJN = DJN.rename(columns={'Unnamed: 0': "Date"})
BNZ = BNZ.rename(columns={'Unnamed: 0': "Date"})
BEN = BEN.rename(columns={'Unnamed: 0': "Date"})
TPC = TPC.rename(columns={'Unnamed: 0': "Date"})
DPR = DPR.rename(columns={'Unnamed: 0': "Date"})
WSJ = WSJ.rename(columns={'Unnamed: 0': "Date"})
DWB = DWB.rename(columns={'Unnamed: 0': "Date"})

In [25]:
DJE = DJE.set_index('Date')
BAR = BAR.set_index('Date')
DJN = DJN.set_index('Date')
BNZ = BNZ.set_index('Date')
BEN = BEN.set_index('Date')
TPC = TPC.set_index('Date')
DPR = DPR.set_index('Date')
WSJ = WSJ.set_index('Date')
DWB = DWB.set_index('Date')

In [6]:
def checknews_sc(weight, cap):
    results = {}
    name = ['DJE', 'BAR', 'DJN', 'BNZ', 'BEN', 'TPC', 'DPR', 'WSJ', 'DWB']
    for index, news_c in enumerate([DJE, BAR, DJN, BNZ, BEN, TPC, DPR, WSJ, DWB]):
        news_c = news_c.fillna(0)
        avg_scores = test.five_days_avgsc(news_c)
        weig_scores =  test.five_days_weight(news_c, weight)
        sta2 = test.test_strategy2(avg_scores, cap)
        sta3 = test.test_strategy2(news_c, cap)
        sta4 = test.test_strategy2(weig_scores, cap)
        #results[name[index]] = [test.getsharp(sta2), test.getsharp(sta3), test.getsharp(sta4)]
        results[name[index]] = [sta2, sta3, sta4]
    #for i in results:
        #print(i)
        #print('5days_avg:',sum(results[i][0].values()))
        #print('pass_day:', sum(results[i][1].values()))
        #print('5_days_weig_scores:', sum(results[i][2].values()))
    return results

## weight [0.5, 0.3, 0.1, 0.07,0.03] cap 0.8

In [27]:
results1 = checknews_sc([0.5, 0.3, 0.1, 0.07,0.03], 0.8)

In [95]:
for i in results1:
    print(i)
    total = 0
    for k in range(3):
        total = sum(test.getsharp(results1[i][k]).values())
        if k == 0:
            print('5days_avg_sum_return:', total)
        if k == 1:
            print('pass_day_sum_return:', total)
        if k == 2:
            print('5_days_weig_scores_sum_return:', total)  

DJE
5days_avg_sum_return: -9.145038163629756
pass_day_sum_return: 1.1290487373897369
5_days_weig_scores_sum_return: -3.804653725523441
BAR
5days_avg_sum_return: 1.9328716445829044
pass_day_sum_return: 23.48766710112351
5_days_weig_scores_sum_return: 0.07439482052524928
DJN
5days_avg_sum_return: 27.016739843198902
pass_day_sum_return: 204.6390302038829
5_days_weig_scores_sum_return: 22.711847881007273
BNZ
5days_avg_sum_return: 15.595200820836835
pass_day_sum_return: 37.50344581800691
5_days_weig_scores_sum_return: 8.419553661628271
BEN
5days_avg_sum_return: -0.39522510615715895
pass_day_sum_return: -0.5778340185698768
5_days_weig_scores_sum_return: -3.0545524107171955
TPC
5days_avg_sum_return: 11.336886633508614
pass_day_sum_return: 42.50317440092227
5_days_weig_scores_sum_return: -10.604463910697834
DPR
5days_avg_sum_return: 17.247612183084367
pass_day_sum_return: 61.89070424048699
5_days_weig_scores_sum_return: 0.916915754423737
WSJ
5days_avg_sum_return: -0.049764676294735355
pass_day

## weight [0.8, 0.1, 0.05, 0.03,0.02] cap 0.9

In [28]:
results2 =checknews_sc([0.8, 0.1, 0.05, 0.03,0.02], 0.9)

In [99]:
for i in results2:
    print(i)
    total = 0
    for k in range(3):
        total = sum(test.getsharp(results2[i][k]).values())
        if k == 0:
            print('5days_avg_sum_return:', total)
        if k == 1:
            print('pass_day_sum_return:', total)
        if k == 2:
            print('5_days_weig_scores_sum_return:', total)  

DJE
5days_avg_sum_return: -8.476221156042214
pass_day_sum_return: -0.01196673755453781
5_days_weig_scores_sum_return: -5.987566968471712
BAR
5days_avg_sum_return: 2.88857263488653
pass_day_sum_return: 25.138942316776635
5_days_weig_scores_sum_return: 1.4799375966331123
DJN
5days_avg_sum_return: 21.716560193898307
pass_day_sum_return: 202.19882069006135
5_days_weig_scores_sum_return: 89.80815753968274
BNZ
5days_avg_sum_return: 14.886242395361181
pass_day_sum_return: 38.37837908282267
5_days_weig_scores_sum_return: 13.187354553206806
BEN
5days_avg_sum_return: -0.5091682801637774
pass_day_sum_return: -0.5789715496442507
5_days_weig_scores_sum_return: 0.2885337104177077
TPC
5days_avg_sum_return: 9.9012732010618
pass_day_sum_return: 42.94153082531745
5_days_weig_scores_sum_return: 6.545197839880492
DPR
5days_avg_sum_return: 19.75367036414476
pass_day_sum_return: 63.536905871784136
5_days_weig_scores_sum_return: 13.459296347338224
WSJ
5days_avg_sum_return: 1.829222716396661
pass_day_sum_retu

## weight [0.6, 0.2, 0.1, 0.07,0.03] cap 0.7

In [29]:
results3 =checknews_sc([0.6, 0.2, 0.1, 0.07,0.03], 0.7)

In [100]:
for i in results3:
    print(i)
    total = 0
    for k in range(3):
        total = sum(test.getsharp(results3[i][k]).values())
        if k == 0:
            print('5days_avg_sum_return:', total)
        if k == 1:
            print('pass_day_sum_return:', total)
        if k == 2:
            print('5_days_weig_scores_sum_return:', total)  

DJE
5days_avg_sum_return: -4.1359628764189775
pass_day_sum_return: 1.005039718360527
5_days_weig_scores_sum_return: -7.255182421550109
BAR
5days_avg_sum_return: 3.219419093444039
pass_day_sum_return: 23.10240641174322
5_days_weig_scores_sum_return: 1.5576899131356714
DJN
5days_avg_sum_return: 29.331785405917277
pass_day_sum_return: 204.34508378801888
5_days_weig_scores_sum_return: 60.828863911062086
BNZ
5days_avg_sum_return: 10.27734043956617
pass_day_sum_return: 36.99288631999129
5_days_weig_scores_sum_return: 12.788170013176469
BEN
5days_avg_sum_return: -1.9176580585338106
pass_day_sum_return: -0.5985163882171315
5_days_weig_scores_sum_return: -0.7379926846973301
TPC
5days_avg_sum_return: 17.79893736644902
pass_day_sum_return: 42.71192415338405
5_days_weig_scores_sum_return: 5.703314613701161
DPR
5days_avg_sum_return: 9.69814336570068
pass_day_sum_return: 67.09025338754266
5_days_weig_scores_sum_return: 11.43142963315683
WSJ
5days_avg_sum_return: 9.04601924724166
pass_day_sum_return:

## Appling Novelty

In [8]:
from math import log
#news_frame = pd.read('All_news.csv')
start_t = time.time()
all_score = {}
Source = ['DJE', 'BAR', 'DJN', 'BNZ', 'BEN', 'TPC', 'DPR', 'WSJ','DWB']
#counter = 0
for thenewsc in Source:
    start_each = time.time()
    news_date = list(set(news_frame['Date']))
    news_date.sort()
    tstart = time.time()
    for each_day in news_date:
        #counter += 1
        the_day = news_frame.loc[news_frame['adj_date'] == each_day,['Ticker','Sentiment(trinary score)','Confidence','Source', 'Novelty']]
        source_news = the_day.loc[the_day['Source'] == thenewsc]
        valid = source_news.loc[the_day['Sentiment(trinary score)'] != 0]
        the_dict = {}
        for i in range(len(valid)):
            item = valid.iloc[i]
            Novelty = item['Novelty']
            if np.isnan(Novelty):
                Novelty = 1
            ticker = item['Ticker']
            if ticker in the_dict:
                the_dict[ticker] += item['Sentiment(trinary score)']*item['Confidence']*(1/log(Novelty+1))
            else:
                the_dict[ticker] = item['Sentiment(trinary score)']*item['Confidence']*(1/log(Novelty+1))
        all_score[each_day] = the_dict
        #if counter%365 == 0:
            #print(each_day)
            #tend=time.time()
            #print(tend - tstart)
    news_scores = pd.DataFrame(all_score).T
    news_scores = news_scores.rename(columns={'Unnamed: 0': "Date"})
    news_scores.to_csv('news_scores_{}_nov.csv'.format(thenewsc))
    print(time.time() - start_each)
print(time.time() - start_t)

132.02261996269226
141.89590501785278
516.8836596012115
203.51533031463623
139.55961275100708
350.9915404319763
199.88295555114746
161.59178614616394
188.27893137931824
2034.6413023471832


## Different news file and Novelty

In [4]:
DJE =pd.read_csv('news_scores_DJE_nov.csv')
BAR =pd.read_csv('news_scores_BAR_nov.csv')
DJN =pd.read_csv('news_scores_DJN_nov.csv')
BNZ =pd.read_csv('news_scores_BNZ_nov.csv')
BEN =pd.read_csv('news_scores_BEN_nov.csv')
TPC =pd.read_csv('news_scores_TPC_nov.csv')
DPR =pd.read_csv('news_scores_DPR_nov.csv')
WSJ =pd.read_csv('news_scores_WSJ_nov.csv')
DWB =pd.read_csv('news_scores_DWB_nov.csv')
DJE = DJE.rename(columns={'Unnamed: 0': "Date"})
BAR = BAR.rename(columns={'Unnamed: 0': "Date"})
DJN = DJN.rename(columns={'Unnamed: 0': "Date"})
BNZ = BNZ.rename(columns={'Unnamed: 0': "Date"})
BEN = BEN.rename(columns={'Unnamed: 0': "Date"})
TPC = TPC.rename(columns={'Unnamed: 0': "Date"})
DPR = DPR.rename(columns={'Unnamed: 0': "Date"})
WSJ = WSJ.rename(columns={'Unnamed: 0': "Date"})
DWB = DWB.rename(columns={'Unnamed: 0': "Date"})
DJE = DJE.set_index('Date')
BAR = BAR.set_index('Date')
DJN = DJN.set_index('Date')
BNZ = BNZ.set_index('Date')
BEN = BEN.set_index('Date')
TPC = TPC.set_index('Date')
DPR = DPR.set_index('Date')
WSJ = WSJ.set_index('Date')
DWB = DWB.set_index('Date')

## weight [0.5, 0.3, 0.1, 0.07,0.03] cap 0.8

In [8]:
result4 = checknews_sc([0.5, 0.3, 0.1, 0.07,0.03], 0.8)

In [11]:
for i in result4:
    print(i)
    total = 0
    for k in range(3):
        total = sum(test.getsharp(result4[i][k]).values())
        if k == 0:
            print('5days_avg_sum_return:', total)
        if k == 1:
            print('pass_day_sum_return:', total)
        if k == 2:
            print('5_days_weig_scores_sum_return:', total)  

DJE
5days_avg_sum_return: -2.391472027611382
pass_day_sum_return: 2.4757427108049805
5_days_weig_scores_sum_return: -0.4000000000000001
BAR
5days_avg_sum_return: 3.5228851709862927
pass_day_sum_return: 19.535027905476117
5_days_weig_scores_sum_return: 0.9000824968623695
DJN
5days_avg_sum_return: 26.454886396712194
pass_day_sum_return: 57.53127887896334
5_days_weig_scores_sum_return: 19.61362326133689
BNZ
5days_avg_sum_return: 2.698113089719284
pass_day_sum_return: 38.69662126882171
5_days_weig_scores_sum_return: 10.63759271347923
BEN
5days_avg_sum_return: -0.9560506536175685
pass_day_sum_return: -0.8723649563111069
5_days_weig_scores_sum_return: -1.589045858906295
TPC
5days_avg_sum_return: 19.287295374409034
pass_day_sum_return: 43.65160704378939
5_days_weig_scores_sum_return: 2.4281001669405464
DPR
5days_avg_sum_return: 10.38650483374119
pass_day_sum_return: 53.846303794781626
5_days_weig_scores_sum_return: -1.0866120444299072
WSJ
5days_avg_sum_return: 4.866956285227649
pass_day_sum_r

## weight [0.8, 0.1, 0.05, 0.03,0.02], 0.9

In [13]:
result5 = checknews_sc([0.8, 0.1, 0.05, 0.03,0.02], 0.9)

In [15]:
for i in result5:
    print(i)
    total = 0
    for k in range(3):
        total = sum(test.getsharp(result5[i][k]).values())
        if k == 0:
            print('5days_avg_sum_return:', total)
        if k == 1:
            print('pass_day_sum_return:', total)
        if k == 2:
            print('5_days_weig_scores_sum_return:', total)  

DJE
5days_avg_sum_return: -6.374370453911575
pass_day_sum_return: 1.5581946666403421
5_days_weig_scores_sum_return: -5.241630970886998
BAR
5days_avg_sum_return: 3.335417300396425
pass_day_sum_return: 21.043843038606205
5_days_weig_scores_sum_return: 3.6672367153218777
DJN
5days_avg_sum_return: 22.153955089145278
pass_day_sum_return: 57.76080847915578
5_days_weig_scores_sum_return: 69.97309700823617
BNZ
5days_avg_sum_return: 10.42000204387031
pass_day_sum_return: 38.2197161048807
5_days_weig_scores_sum_return: 23.283430672372067
BEN
5days_avg_sum_return: -1.5261633471281635
pass_day_sum_return: -0.5111043516880844
5_days_weig_scores_sum_return: 0.19607388907590217
TPC
5days_avg_sum_return: 18.21627331103819
pass_day_sum_return: 42.216033412114236
5_days_weig_scores_sum_return: 6.098201456713387
DPR
5days_avg_sum_return: 11.595041716542788
pass_day_sum_return: 88.73072771433095
5_days_weig_scores_sum_return: 4.540099971298237
WSJ
5days_avg_sum_return: 1.1808532941454404
pass_day_sum_retu

## weight [0.6, 0.2, 0.1, 0.07,0.03], 0.7

In [14]:
result6 = checknews_sc([0.6, 0.2, 0.1, 0.07,0.03], 0.7)

### Events Return

AA@RAT
5days_avg: -3.598363606633649
pass_day: 57.66048616097324
5_days_weig_scores: -11.999262372174575
AA@GOV
5days_avg: -5.379624359155068
pass_day: 7.250665830304889
5_days_weig_scores: -6.825520598967216
AA@PRE
5days_avg: -21.52506910549048
pass_day: -2.192191063239607
5_days_weig_scores: -19.22855438679025
AA@EST
5days_avg: -1.8074680441117505
pass_day: 162.35156711958442
5_days_weig_scores: -16.79783043754806
AA@INS
5days_avg: 14.016086958845316
pass_day: 65.2118487198316
5_days_weig_scores: -0.8729533654340997
AA@IPO
5days_avg: -0.38961773851170345
pass_day: 6.861738194792907
5_days_weig_scores: -13.504275071303942
AA@DRV
5days_avg: -1.4893216124085233
pass_day: -6.913706265066142
5_days_weig_scores: -0.4000000000000001
AA@CBT
5days_avg: 0.15362559725469305
pass_day: 1.4337995124829657
5_days_weig_scores: -3.844131458245129
AA@MFD
5days_avg: -8.80984203091941
pass_day: 1.9888688117545783
5_days_weig_scores: -18.869068060349953
AA@BNQ
5days_avg: -2.132438074718158
pass_day: -2.2065447716616844
5_days_weig_scores: -3.059951691106284
AA@FDA
5days_avg: 8.051422476368652
pass_day: 34.39778426835123
5_days_weig_scores: 4.916945161627319
AA@ALEX
5days_avg: 28.20382590087018
pass_day: 169.6490797539249
5_days_weig_scores: 94.51169993683952
AA@OPS
5days_avg: 11.740002239739217
pass_day: 43.546438836817266
5_days_weig_scores: 5.4853705746952315
AA@CPG
5days_avg: -3.8025670008320813
pass_day: 1.8849752250811609
5_days_weig_scores: -14.423078594712251
AA@HFD
5days_avg: 2.885792921780886
pass_day: 12.574835703866842
5_days_weig_scores: -15.815838268729529
AA@MGT
5days_avg: -6.5735839040354325
pass_day: 9.546880914532096
5_days_weig_scores: -99.06381612779204
AA@GDC
5days_avg: 19.065066823637984
pass_day: 918.933341247287
5_days_weig_scores: 13.382423691336443
AA@LGL
5days_avg: -11.576450703919479
pass_day: 1.6476586612197697
5_days_weig_scores: -114.06588331055302
AA@REG
5days_avg: -5.147439113124449
pass_day: 11.295433047433168
5_days_weig_scores: -11.394002570301108
AA@CPA
5days_avg: -1.928375487482936
pass_day: 1.0427327054996427
5_days_weig_scores: -2.6118193891101993
AA@BUY
5days_avg: 26.712009790256424
pass_day: 86.95747943918568
5_days_weig_scores: 11.817475272196267
AA@ETF
5days_avg: -0.4000000000000001
pass_day: -21.90859381694642
5_days_weig_scores: -0.4000000000000001
    
    
AA@OWN
5days_avg: -112.55382717931693
pass_day: 27.047013769709885
5_days_weig_scores: -7.238557563028973
AA@ERN
5days_avg: 30.536464640377464
pass_day: 264.9855212127409
5_days_weig_scores: 92.44339544581067
AA@AST
5days_avg: -6.796296489159715
pass_day: -28.20511308111245
5_days_weig_scores: -21.295444822944606
AA@IMB
5days_avg: -6.688552358542846
pass_day: -3.8733575319171543
5_days_weig_scores: -1.7876143338711734
AA@DEB
5days_avg: -2.6284236495170306
pass_day: 6.8772524281624845
5_days_weig_scores: -13.849472069227495
AA@TRD
5days_avg: 6.271702786759633
pass_day: 0.3451519822868967
5_days_weig_scores: -7.95635610918359
AA@ACC
5days_avg: -3.8626740375942394
pass_day: 1.8825587402972517
5_days_weig_scores: -10.304655344384443
AA@MNA
5days_avg: -8.50651681092274
pass_day: 7.380016665121583
5_days_weig_scores: -7.671674701545615
AA@DIV
5days_avg: 17.569948391067406
pass_day: 78.92495018249357
5_days_weig_scores: 10.02634618547885